In [1]:
# from tensorflow.keras.utils import to_categorical
# from tensorflow.keras.models import Model, Sequential
# from tensorflow.keras.layers import Input, Dense, Conv2D, BatchNormalization, Flatten, Softmax, Concatenate, Reshape
# from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.initializers import HeNormal
# import tensorflow as tf
import numpy as np

In [97]:
# Branch:
# Input  = board state
# Output = Logits

input_layer = Input(shape=(7, 7, 1))

# Phân tích insight
conv = Conv2D(256, (3, 3), activation='linear', padding='same', kernel_initializer=HeNormal())(input_layer)
conv = Conv2D(512, (3, 3), activation='linear', padding='same', kernel_initializer=HeNormal())(conv)
insight = Flatten()(conv)
insight = BatchNormalization()(insight)

# Merge: insight + current_state
state = Flatten()(input_layer)
dense_input = Concatenate()([insight, state])

dense_input = BatchNormalization()(dense_input)
# Xử lý chính
dense = Dense(512, activation="linear", kernel_initializer='he_normal')(dense_input)
dense = BatchNormalization()(dense)
dense = Dense(256, activation="linear", kernel_initializer='he_normal')(dense)
dense = BatchNormalization()(dense)
dense = Dense(64, activation="linear", kernel_initializer='he_normal')(dense)
dense = BatchNormalization()(dense)
dense = Dense(7, activation="tanh")(dense)  


Branch = Model(inputs=input_layer, outputs=dense)       # => Logits


Branch.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=1e-4), metrics=['mse', 'accuracy'])

# Branch.summary()

In [ ]:
# Root:
# Input  = concatenate([Branch])
# Output = Softmax

logits = Branch(input_layer)
probabilities = Softmax()(logits)

Root = Model(inputs=input_layer, outputs=probabilities)   # Thực hiện Softmax trực tiếp lên output của Branch

Root.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=1e-4), metrics=['accuracy'])

In [3]:
train_data = np.load('data/Train_RandomizeMinimax.npy')

label_data = np.load('data/Label_RandomizeMinimax.npy')

# for i in range(len(label_data)) :
#     argmax = np.argmax(label_data[i])
#     label_data[i] = np.zeros(shape=(7, ))
#     label_data[i][argmax] = 1

if len(train_data) == len(label_data) :
    print(len(train_data), 'samples')
    print('train_shape=', train_data.shape)
    print('label_shape=', label_data.shape)
    print(train_data[-1])
    print(label_data[-1])
else :
    'Lỗi về mẫu training-set'

4752 samples
train_shape= (4752, 7, 7)
label_shape= (4752, 7)
[[ 0. -1.  0.  1.  0.  0.  0.]
 [ 0. -1.  0. -1. -1. -1.  0.]
 [ 0.  1.  0.  1.  1. -1.  0.]
 [ 0.  1. -1. -1. -1.  1.  0.]
 [ 0. -1.  1.  1.  1. -1.  0.]
 [ 0.  1. -1. -1.  1.  1.  0.]
 [ 0. -1.  1.  1.  1. -1.  0.]]
[0. 0. 1. 0. 0. 0. 0.]


In [98]:
Branch.fit(train_data, label_data, epochs = 10, verbose = 1)

Epoch 1/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 16s 98ms/step - accuracy: 0.1413 - loss: 6.6074 - mse: 0.7952
Epoch 2/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 15s 99ms/step - accuracy: 0.1435 - loss: 5.7201 - mse: 0.8458
Epoch 3/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 14s 96ms/step - accuracy: 0.1525 - loss: 5.5455 - mse: 0.8566
Epoch 4/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 14s 96ms/step - accuracy: 0.1507 - loss: 5.2795 - mse: 0.8621
Epoch 5/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 14s 96ms/step - accuracy: 0.1588 - loss: 5.1516 - mse: 0.8621
Epoch 6/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 14s 97ms/step - accuracy: 0.1657 - loss: 5.0514 - mse: 0.8680
Epoch 7/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 15s 98ms/step - accuracy: 0.1471 - loss: 4.5339 - mse: 0.8730
Epoch 8/10
108/148 ━━━━━━━━━━━━━━━━━━━━ 3s 99ms/step - accuracy: 0.1520 - loss: 4.3989 - mse: 0.8667

KeyboardInterrupt: 

In [100]:
Branch.predict(np.array([[[0, 0, 0, 0, 0, 0, 0],
                         [0, 0, 0, 0, 0, 0, 0],
                         [0, 0, 0, 0, 0, 0, 0],
                         [0, 0, 0, 0, 0, 0, 0],
                         [0, 0, -1, -1, 0, 0, 0],
                         [0, 0, -1, 1, -1, 0, 0],
                         [0, 0, -1, 1, 1, 0, 0]]]))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step


array([[-0.99662024, -0.5662685 , -0.94979405, -0.33175218,  0.9731075 ,
        -0.24358462,  0.317431  ]], dtype=float32)

In [ ]:
Root.save("DL/model/Root.h5")
Branch.save("DL/model/Branch.h5")